In [34]:
news_data = []
with open("news_summary.txt", "r", encoding="utf-8") as file:
    news_data = file.readlines()
    print(f"loaded {len(news_data)}")

loaded 259


In [35]:
import ollama
embedding_model = "hf.co/CompendiumLabs/bge-base-en-v1.5-gguf"
language_model = "qwen3:latest" # qwen is goated going through 
 
vector_db = []

In [36]:
def add_chunk_to_database(chunk):
    embedding = ollama.embed(model=embedding_model, input=chunk)["embeddings"][0]
    vector_db.append((chunk, embedding))

In [28]:
for i, chunk in enumerate(news_data):
    add_chunk_to_database(chunk)
    print(f"added chunk {i+1}/{len(news_data)} to the database")

added chunk 1/178 to the database
added chunk 2/178 to the database
added chunk 3/178 to the database
added chunk 4/178 to the database
added chunk 5/178 to the database
added chunk 6/178 to the database
added chunk 7/178 to the database
added chunk 8/178 to the database
added chunk 9/178 to the database
added chunk 10/178 to the database
added chunk 11/178 to the database
added chunk 12/178 to the database
added chunk 13/178 to the database
added chunk 14/178 to the database
added chunk 15/178 to the database
added chunk 16/178 to the database
added chunk 17/178 to the database
added chunk 18/178 to the database
added chunk 19/178 to the database
added chunk 20/178 to the database
added chunk 21/178 to the database
added chunk 22/178 to the database
added chunk 23/178 to the database
added chunk 24/178 to the database
added chunk 25/178 to the database
added chunk 26/178 to the database
added chunk 27/178 to the database
added chunk 28/178 to the database
added chunk 29/178 to the dat

In [29]:
# use cosine similiarity to find close chunks 
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

In [30]:
def retrieve(query, top_n=10):
    query_embedding = ollama.embed(model=embedding_model, input=query)["embeddings"][0]
    similarities = []
    for chunk, embedding in vector_db:
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((chunk, similarity))
    # sort by similarity in descending order, because higher similarity means more relevant chunks
    similarities.sort(key=lambda x: x[1], reverse=True)
    # finally, return the top N most relevant chunks
    return similarities[:top_n]

In [31]:
input_query = input("Ask me anything about stocks! ")
retrieved_knowledge = retrieve(input_query)

print("retrieved knowledge: ")
for chunk, similarity in retrieved_knowledge:
    print(f" - (similarity: {similarity:.2f}) {chunk}")

context_text = '\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])

instruction_prompt = f"""
You are a stock market analysis bot with the personality of a full-blown WallStreetBets degenerate. 
You have access to real-time news via RAG and must always ground your takes in the retrieved articles. 

Rules:
1. Use ONLY the retrieved news articles as your source of information.
2. Always rely on the LATEST news provided in the feed. Do not invent or fabricate numbers, prices, or events.
3. Respond in ONE long chaotic paragraph like a WSB rant. 
   - No bullet points, no lists, no neat formatting.
   - Make it loud, funny, and dripping with degeneracy.
4. Swear freely, roast companies, mock bagholders, and lean hard into WSB slang and memes 
   (diamond hands 💎🙌, paper hands, stonks 📈, YOLO, 🚀🌕, “send it,” “bagholder tears,” “tendies”).
5. Facts from the news must still be included, but bury them inside the chaos. 
   Example: “According to Bloomberg, Apple printed money again, but who gives a shit because I sold at the bottom like a smooth-brained idiot.”
6. If the news is boring, trash it like it wasted your time. 
   If it’s hype, crank the degeneracy up and scream like the stock is going parabolic.
7. Do NOT sound professional, analytical, or polite. 
   You are a meme-spewing gambler pretending to care about fundamentals while actually YOLOing weekly options.

Your goal is to return a single, unfiltered, profanity-laced, meme-heavy rant that still sneaks in the actual news but feels like pure WallStreetBets chaos.
{context_text}
"""

retrieved knowledge: 
 - (similarity: 0.64) Koss Olinger Consulting LLC increased its stake in Microsoft Corporation (NASDAQ:MSFT – Free Report) by 36.8% in the 1st quarter, Holdings Channel reports. Koss Olinger Consulting LLC’s holdings in Microsoft were worth $5,781,000 at the end of the most recent reporting period. Geode Capital Management LLC now owns 168,128,024 shares of the software giant’s stock valued at $70,720,874,000 after buying an additional 3,103,212 shares during the period. Norges Bank purchased a new stake in Microsoft during the fourth quarter valued at about $44,343,058,000. Following the completion of the sale, the chief executive officer directly owned 790,852 shares in the company, valued at $399,206,272.56.

 - (similarity: 0.66) NVIDIA Corporation (NASDAQ:NVDA) is one of the Best Reddit Stocks to Invest in Now. On September 4, Reuters reported that Alibaba, ByteDance, and other Chinese tech firms are keen on NVIDIA Corporation (NASDAQ:NVDA)’s AI chips even th

In [11]:
stream = ollama.chat(
    model=language_model,
    messages=[
        {'role': 'system', 'content': instruction_prompt},
        {'role': 'user', 'content': input_query},
    ],
    stream=True
)

print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Chatbot response:
<think>
Okay, let's tackle this query. The user wants news on NVIDIA. First, I need to go through all the provided articles and extract the key points. 

Looking at the first article, there's mention of Selway Asset Management buying NVIDIA stock for $269k, and the average rating is "Moderate Buy" with a price target of $207.69. The stock is down 2.7% and has moving averages at $172.73 and $140.21. Also, NVIDIA's profile is mentioned, but it's a bit generic.

Next, there's a Reuters report about Chinese tech firms like Alibaba and ByteDance still interested in NVIDIA's AI chips despite regulatory issues. They need reassurance about H20 model orders. Also, Trump's deal for 15% H20 revenue for the US government. NVIDIA's Q3 2026 revenue is expected at $54B, excluding H20 China shipments.

Then, Wall Street Zen upgraded to "buy," Arete Research to "strong-buy," and some insider sales by Tench Coxe and Jen Hsun Huang. Hedge funds are active, and some cut or increased stak